In [ ]:
!pip install langchain_community
!pip install langchain_openai
!pip install wikipedia
!pip install faiss-cpu
!pip install openai

In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

api_key=" " # write down your real api key

os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def build_example_text():
    return """
Example of CE reasoning (Korean Folktale Style):

Original: Once upon a time, a wandering traveler came to a lively village market.
The place was bustling and cheerful — drums beat loudly,
and people gathered together, wishing each other happiness.

Someone kindly offered the traveler a cup of wine.
He took a small sip and thought,
“Ah, life is happiest when people care for one another.”

The market was filled with bright laughter and warm smiles,
and both children and grown-ups enjoyed the day together.

Moral: When we share and help one another,
the world becomes a joyful place for everyone.

Reasoning:
1. Culturally Specific Items (CSI):
- cowboy
- wine
- church festival

2. Replacements (Korean folktale style):
- cowboy → 떠돌이 나그네 (a wanderer; in Korean folktales, such characters symbolize wisdom from travels)
- wine → 막걸리 (makgeolli; traditional Korean rice wine, often shared at communal events)
- church festival → 장터/마을 굿 (village market or shamanic gut ritual, common gathering scenes)

3. Result (Korean Folktale Style):
옛날 옛적에, 한 떠돌이 나그네가 고을 장터에 찾아왔단다.
그날 장터는 북적북적, 꽹과리 소리 쿵쾅쿵쾅 울려 퍼지고
사람들은 마을굿을 하며 서로의 행복을 빌었지.

나그네는 따뜻한 마음씨로 건네받은 막걸리를
조심조심 한 모금 마시고는,
‘아, 사람은 서로 도우며 살아야 하는구나’ 하고 생각했단다.

그날의 장터는 웃음과 정(情)으로 가득 차
아이도 어른도 모두 신나게 웃고 즐겼더라.

이렇게, 서로 나누고 도우면 세상은 언제나 즐겁고 따뜻해진단다.
""".strip()

def build_cot_ce_se_prompt(text, cultural_context=""):
    example_hint = build_example_text()

    example_char_names = ['이 도령','개똥 어머니', '개똥이', '게으름뱅이', '견우', '그릇 장수', '김 진사의 딸', '김첨지', '김홍도', '꼬부랑 할아버지', '나그네', '나무꾼','남생이', '노인', '노인 부부', '놀부', '대감', '덕돌이', '덕순', '덕이', '덕희', '동이', '사공', '떡보', '똘똘', '모란꽃', '모자 만드는 할아버지' '몽순이', '몽순이 동생', '무학대사', '뭉태', '민이', '바람', '바보', '박 선비', '박 의원', '박문수', '반쪽이', '밥벌레 장군', '방울 사또', '백주', '범아재비', '별당할멈', '보따리장수', '복녀', '복녀의 남편', '복만이', '봇짐장수', '봉구', '부모님', '부부', '부인', '부자', '부자 영감', '부자 할아버지', '부잣집 딸', '비단 장수', '비단장수', '뺑덕어멈', '사기장수', '사냥꾼', '사또', '산신령', '삼돌이', '상주', '선녀', '설문대할망', '성서방네 처녀', '소금장수', '솔개', '쇠돌 엄마', '수재', '순덕이', '순돌', '순돌이 엄마', '순이','심술쟁이 혹부리 할아버지', '심청', '심학규', '청개구리', '아름다운 여인', '앉은뱅이', '양근댁', '양근댁 남편', '얼벙이', '여염집 아낙', '연꽃', '영득', '영식', '오늘이', '옥이', '옹고집', '왕눈이', '왕달', '왕방울', '왕서방', '욕심쟁이', '우렁각시', '우리 수탉', '원님', '이 도령', '이 진사', '이무기', '이방', '이성계', '이주사', '입 큰 각시', '잉어색시', '점순', '점순이', '직녀', '혹부리 할아버지', '처녀', '척척꼼꼼댁', '천 서방', '춘호', '충줏집', '치삼', '해님', '훈이', '흥부']


    name_reference_note = f"""
Names in Korean folktales are usually role-based, virtue-based, or relational.
Reference list: {', '.join(example_char_names)}.
"""

    return f"""
Translate the following English text into **Korean** in **traditional Korean folktale style**.

1. **CSI Identification**
   - Identify all culturally specific items (CSI) in the input text that are inappropriate in Korean folktales.
2. **Cultural Adaptation**
   - Replace each CSI with a culturally appropriate **Korean traditional equivalent**.
   - Use the following Wikipedia-derived cultural context to INCREASE CULTURAL APPROPRIATENESS of your generation.
     Ground concrete details (clothing, food, region festivals, Religion, social customs) the following **Wikipedia-derived cultural context**:
     {cultural_context}
   - For names and roles, refer to the following **Korean folktale name reference list**:
     {name_reference_note}
3. **Folktale-style Generation**
   - Following the pattern in this **example reasoning**:
     {example_hint}
   - Produce the final story in Korean folktale style, concise, rhythmic, and oral.
   - Use only Joseon-era cultural and narrative elements, with no modern intrusions.
**Input Text:**
{text}
""".strip()

def convert_folktale_cot_ce_se(text, cultural_context=""):
    prompt = build_cot_ce_se_prompt(text, cultural_context=cultural_context)

    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are an AI storyteller who transforms global fairy tales into Korean traditional folktale style, considering cultural transfer with CEreasoning. Do not generate sentences that are too long. Keep sentences concise and readable."},
            {"role": "user", "content": prompt}
        ],
        )
    return response.choices[0].message.content

    from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

topics = [
    "Culture of Korea",
    "Korean architecture",
    "Korean Buddhism",
    "Korean shamanism",
    "Korean shamanism",
    "Korean cuisine",
    "List of Korean clothing",
    "Korean New Year",
    "Chuseok"
]

docs = []
for t in topics:
    try:
        docs.extend(WikipediaLoader(query=t, lang="en").load())
    except Exception as e:
        print(f"[WARN] Failed to load {t}: {e}")

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=os.environ["OPENAI_API_KEY"])
db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 8})

if __name__ == "__main__":
    english_text = """
A long time ago, a child was born to a queen and king and she was called Snow White.  When the queen died, the king married again.  This new queen was wicked and hated Snow white.  The queen gave orders that Snow White was to be treated as a servant.
Snow White grew very beautiful and one day a Prince riding by, saw her at work and fell in love with her.
The queen was beautiful too, and every day she asked her Magic Mirror, "Who is the fairest in the land?" and the mirror always answered, "You are the fairest one of all".
But one day the mirror answered Snow White was the fairest in the land, and in a rage the queen gave orders to one of her Huntsmen to take Snow White into the woods and kill her.
The Huntsman had a kind heart and couldn't do the deed so told her to run away.  She fled into the woods where Seven little dwarfs lived.  Their house was small and strange.

 Snow White entered the little house and finding it very untidy, started to clean up.  Upstairs she found seven little beds.  She was very tired and stretching out on one of the beds, was soon asleep.
When the Dwarfs came home they were surprised to find Snow White and after some argument, decided to let her stay.  She promised to cook and look after them.
The Queen discovered where Snow White was living and disguising herself as a witch, took a poisoned apple and set out for the Dwarfs cottage.  She gave Snow White the poisoned apple to eat and as soon as she bit the apple, she sank into unconsciousness.
Thinking she was dead, the Dwarfs built a glass coffin and put her in it. For days she lay in the forest in her glass coffin.  One day, the Prince was riding through the forest looking for Snow White and found her.  He leaned over and kissed her.  She opened her eyes and sat up with a smile. Everyone was happy.  The Prince took Snow White to his palace where they were married and lived happily ever after.

"""


    query = "Clothing Cuisine General characteristics Religious architecture Pansori Ceremonial, ritual and temple foods Royal court cuisine (surasang) Festivals Ceramics Houses Religious beliefs Influence of Buddhism on culture and traditions "
    results = retriever.invoke(query)
    cultural_context = "\n".join([r.page_content for r in results])
    for i, r in enumerate(results, 1):
        print(f"Chunk {i}: {r.page_content[:200]}...\n")


    output = convert_folktale_cot_ce_se(english_text, cultural_context)
    print("=== Final Folktale Output ===")
    print(output)